# Joint Task Post-Processing

## Libraries

In [ ]:
import os
import re
import pickle
import json
import pandas as pd
import ast

from sklearn.metrics import classification_report

In [2]:
#am_work/coling_2025/cdcp/finetuned_models_run3/CDCP_acc_mistral-7b-instruct-v0.3-bnb-4bit

In [3]:
# nb_epochs = 0.2
# OUTPUT_DIR = "../finetuned_models_run3/CDCP_acc_mistral-7b-instruct-v0.3-bnb-4bit"

In [4]:
with open("/Utilisateurs/umushtaq/am_reasoning/cdcp/saved_models/cdcp_pipeline_llama-3-8b-Instruct-bnb-4bit/cdcp_pipeline_results_5.pickle", "rb") as fh:
        
        results = pickle.load(fh)

In [5]:
results

{'grounds': ['### Output:\n{"argument_types": [\'F\', \'V\', \'P\']}\n{"relation_types": [(3, 2, \'R\')]}\n',
  '### Output:\n{"argument_types": [\'V\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'P\', \'P\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'V\', \'V\']}\n{"relation_types": []}\n',
  '### Output:\n{"argument_types": [\'V\', \'V\', \'V\', \'T\', \'V\']}\n{"relation_types": [(5, 3, \'R\'), (5, 4, \'R\')]}\n',
  '### Output:\n{"argument_types": [\'T\', \'T\', \'T\', \'V\', \'V\', \'P\']}\n{"relation_types": [(5, 1, \'R\'), (5, 2, \'R\'), (5, 3, \'R\'), (5, 4, \'R\')]}\n',
  '### Output:\n{"argument_types": [\'V\', \'F\', \'T\', \'T\', \'T\']}\n{"relation_types": [(2, 3, \'E\'), (2, 4, \'E\'), (2, 5, \'E\'), (1, 4, \'R\'), (1, 5, \'R\')]}\n',
  '### Output:\n{"argument_types": [\'V\', \'T\', \'V\', \'V\', \'P\', \'P\']}\n{"relation_types": [(1, 2, \'R\'), (4, 3, \'R\'), (6, 3, \'R\')]}\n',
  '### Output:\n

In [6]:
grounds_global = results["grounds"]

In [7]:
preds_global = results["predictions"]

In [8]:
len(grounds_global), len(preds_global)

(150, 150)

## ACC post-processing

In [30]:
grounds_global[0].split("\n")[1]

'{"argument_types": [\'F\', \'V\', \'P\']}'

In [31]:
grounds_acc = [json.loads(x.replace('### Output:\n', '').strip().split('\n')[0].replace("'", '"'))["argument_types"] for x in grounds_global]

In [32]:
predictions_acc = [json.loads(x.replace('### Output:\n', '').strip().split('\n')[0].replace("'", '"'))["argument_types"] for x in preds_global]

In [33]:
len(grounds_acc), len(predictions_acc)

(150, 150)

In [13]:
bad_idx = []
for i,(x,y) in enumerate(zip(grounds_acc, predictions_acc)):
    
    if len(x) != len(y):
        bad_idx.append(i)
        print(i)
        #preds[i] = harmonize_preds(x, y)

75


In [14]:
grounds_acc = [elem for idx, elem in enumerate(grounds_acc) if idx not in bad_idx]
predictions_acc = [elem for idx, elem in enumerate(predictions_acc) if idx not in bad_idx]

In [15]:
ATC_preds = [item for row in predictions_acc for item in row]
ATC_grounds = [item for row in grounds_acc for item in row]

In [16]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

In [18]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

           F      0.614     0.633     0.623       128
           P      0.937     0.930     0.933       143
           R      1.000     1.000     1.000         1
           T      0.908     0.889     0.899       244
           V      0.868     0.872     0.870       476

    accuracy                          0.854       992
   macro avg      0.865     0.865     0.865       992
weighted avg      0.855     0.854     0.854       992



### ARIC

In [34]:
import re

In [35]:
len(grounds_global), len(preds_global)

(150, 150)

In [36]:
grounds_aric = [json.loads(json.dumps(ast.literal_eval(x.replace('### Output:\n', '').strip().split('\n')[1])))["relation_types"] for x in grounds_global]

In [37]:
grounds_aric

[[[3, 2, 'R']],
 [],
 [],
 [],
 [[5, 3, 'R'], [5, 4, 'R']],
 [[5, 1, 'R'], [5, 2, 'R'], [5, 3, 'R'], [5, 4, 'R']],
 [[2, 3, 'E'], [2, 4, 'E'], [2, 5, 'E'], [1, 4, 'R'], [1, 5, 'R']],
 [[1, 2, 'R'], [4, 3, 'R'], [6, 3, 'R']],
 [[4, 5, 'R'], [4, 6, 'R'], [6, 8, 'R'], [6, 9, 'R'], [6, 10, 'R']],
 [[1, 2, 'R'], [1, 4, 'R']],
 [[2, 3, 'R'], [2, 4, 'R'], [2, 5, 'R'], [6, 7, 'R'], [6, 8, 'R']],
 [[3, 4, 'R']],
 [[1, 2, 'R'], [1, 3, 'R'], [5, 2, 'R']],
 [],
 [[2, 3, 'R'], [2, 4, 'R']],
 [[1, 2, 'R'], [3, 4, 'R']],
 [[1, 2, 'R'], [1, 3, 'R'], [1, 4, 'R']],
 [[1, 2, 'R'], [1, 3, 'R'], [1, 4, 'R']],
 [],
 [],
 [[1, 3, 'R'], [3, 2, 'R'], [5, 6, 'R']],
 [[2, 3, 'R'], [2, 4, 'R'], [7, 8, 'R'], [8, 9, 'R']],
 [[3, 4, 'R']],
 [],
 [],
 [[2, 3, 'R'],
  [2, 4, 'R'],
  [2, 5, 'R'],
  [9, 11, 'R'],
  [9, 12, 'R'],
  [9, 10, 'R'],
  [14, 13, 'R'],
  [26, 25, 'R'],
  [26, 27, 'R']],
 [[5, 4, 'R']],
 [],
 [[2, 3, 'E'],
  [2, 4, 'E'],
  [5, 1, 'R'],
  [5, 2, 'R'],
  [6, 7, 'R'],
  [7, 1, 'R'],
  [7, 2, 'R']],

In [38]:
def safe_eval(s):
    try:
        return ast.literal_eval(s)
    except (SyntaxError, ValueError):
        # Fix unquoted 'reason' strings like: (3, 1, reason') → (3, 1, 'reason')
        s = re.sub(r"\((\d+),\s*(\d+),\s*([a-zA-Z_][a-zA-Z0-9_]*)'\)", r"(\1, \2, '\3')", s)
        try:
            return ast.literal_eval(s)
        except Exception as e:
            print("Still failed to parse:", s)
            return {}

In [39]:
predictions_aric = []

for x in preds_global:
    try:
        output_line = x.replace('### Output:\n', '').strip().split('\n')[1]
        parsed = safe_eval(output_line)
        if isinstance(parsed, dict) and "relation_types" in parsed:
            predictions_aric.append(parsed["relation_types"])
        else:
            predictions_aric.append([])  # Fallback for malformed inputs
    except Exception as e:
        print(f"Error parsing line: {x}\n{e}")
        predictions_aric.append([])  # Add empty entry on failure

In [40]:
len(grounds_aric), len(predictions_aric)

(150, 150)

In [41]:
def process_grounds_aric(grounds_l, nr_acs):
    
    relations_aric = []
    
    relation_dict = {(i, j): rel for i, j, rel in grounds_l}
    
    for i in range(1, nr_acs + 1):
        for j in range(1, nr_acs + 1):
            if i != j:
                relation = relation_dict.get((i, j), "NR")
                relations_aric.append((i, j, relation))
                
    return relations_aric

In [42]:
nr_acs_l = [len(elem) for elem in grounds_acc]

In [43]:
ground_relations = []

for i, j in zip(grounds_aric, nr_acs_l):
    ground_relations.append(process_grounds_aric(i, j))

In [44]:
len(ground_relations)

150

In [45]:
def process_predictions_aric(predictions_l, nr_acs):
    
    relations_aric = []
    
    relation_dict = {(i, j): rel for i, j, rel in predictions_l}
    
    for i in range(1, nr_acs + 1):
        for j in range(1, nr_acs + 1):
            if i != j:
                relation = relation_dict.get((i, j), "NR")
                relations_aric.append((i, j, relation))
                
    return relations_aric

In [46]:
prediction_relations = []

for i, j in zip(predictions_aric, nr_acs_l):
    prediction_relations.append(process_predictions_aric(i, j))

In [47]:
len(prediction_relations)

150

In [48]:
grounds_l = [elem[2] for sl in ground_relations for elem in sl]
predictions_l = [elem[2] for sl in prediction_relations for elem in sl]

In [49]:
len(grounds_l), len(predictions_l)

(10328, 10328)

In [50]:
print(classification_report(grounds_l, predictions_l, digits=3))

              precision    recall  f1-score   support

           E      0.000     0.000     0.000        26
          NR      0.980     0.984     0.982     10004
           R      0.422     0.406     0.414       298

    accuracy                          0.964     10328
   macro avg      0.467     0.463     0.465     10328
weighted avg      0.961     0.964     0.963     10328

